In [1]:
import os

In [2]:
os.chdir('../')


In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:

    root_dir: Path
    data_path: Path


In [4]:
from NBA_Project.constants import *
from src.NBA_Project.utils.common import read_yaml,create_directories


In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
        )

        return data_transformation_config

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from src.NBA_Project import logger


class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config


    def transform_data(self):

        data=pd.read_csv(self.config.data_path)
        train,test=train_test_split(data,test_size=0.2,shuffle=True,stratify=data['TARGET_5Yrs'],random_state=42)

        num_col=list(train.drop(['TARGET_5Yrs'],axis=1).select_dtypes(include=['int64','float64']))
        for col in num_col:

                q1=train[col].quantile(0.25)
                q3=train[col].quantile(0.75)
                IQR=q3-q1
                lb=q1-1.5*IQR
                up=q3+1.5*IQR

                train=train[(train[col]>=lb) & (train[col]<=up)]

        imputer=SimpleImputer(strategy='median')

        train["3P%"]=imputer.fit_transform(train[['3P%']])
        test["3P%"]=imputer.transform(test[['3P%']])
        

        train.drop('Name',axis=1,inplace=True)
        test.drop('Name',axis=1,inplace=True)

        train.to_csv(os.path.join(self.config.root_dir, "train.csv"),index = False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"),index = False)

        logger.info("Splited data into training and test sets")
        logger.info(train.shape)
        logger.info(test.shape)

        print(train.shape)
        print(test.shape)




In [8]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.transform_data()
except Exception as e:
    raise e

[2023-10-21 16:39:55,827: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-10-21 16:39:55,830: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-21 16:39:55,833: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-10-21 16:39:55,835: INFO: common: created directory at artifacts]
[2023-10-21 16:39:55,837: INFO: common: created directory at artifacts/data_transformation]
[2023-10-21 16:39:55,942: INFO: 288601790: Splited data into training and test sets]
[2023-10-21 16:39:55,943: INFO: 288601790: (614, 20)]
[2023-10-21 16:39:55,945: INFO: 288601790: (268, 20)]
(614, 20)
(268, 20)
